In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.4 MB/s eta 0:00:00


In [8]:
import pandas as pd
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [9]:
# !pip install transformers torch

In [10]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [14]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Load the IMDB reviews dataset
df_critic = pd.read_csv('/content/rotten_tomatoes_critic_reviews.csv', nrows = 3000)
df_critic = df_critic.dropna()

df  = df_critic[['review_content', 'review_type']]
# Map the labels to 0 (negative) and 1 (positive)
df['review_type'] = df['review_type'].map({'Rotten': 0, 'Fresh': 1})

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['review_content'].to_list(), df['review_type'].to_list(), test_size=0.2)

# Load the BERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Convert the text data to BERT input
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

# Create the BERT model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Convert data to tensors
train_inputs = torch.tensor(train_encodings['input_ids']).to(device)
train_labels = torch.tensor(y_train).to(device)
test_inputs = torch.tensor(test_encodings['input_ids']).to(device)
test_labels = torch.tensor(y_test).to(device)

# Set batch size and create data loaders
batch_size = 16
train_data = torch.utils.data.TensorDataset(train_inputs, train_labels)
train_sampler = torch.utils.data.RandomSampler(train_data)
train_dataloader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
test_data = torch.utils.data.TensorDataset(test_inputs, test_labels)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

# Set optimizer and learning rate
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Training loop
num_epochs = 1
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        batch_inputs, batch_labels = batch
        outputs = model(batch_inputs, labels=batch_labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {average_loss:.4f}")

# Evaluation


<ipython-input-14-f7f935d20127>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_type'] = df['review_type'].map({'Rotten': 0, 'Fresh': 1})
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSeque

Epoch 1/1 - Loss: 0.6795


In [15]:
model.eval()
predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        batch_inputs, batch_labels = batch
        outputs = model(batch_inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1)
        predictions.extend(predicted_labels.cpu().numpy())

# Calculate metrics
accuracy = accuracy_score(y_test, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, predictions, average='binary')

# Print the metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1",f1)


Accuracy: 0.7036144578313253
Precision: 0.7256637168141593
Recall: 0.7288888888888889
F1 0.7272727272727273
